# Data lake access

## Containers

### With a SAS

In [1]:
import pandas as pd

In [ ]:
# Generated SAS for csv in container
sas_token = '<sas_token>'
sas_uri ='<sas_url>'

df = pd.read_csv(sas_uri) 
df

### With SDK - gen2


In [ ]:
# !pip install azure-storage-file-datalake --pre
# https://pypi.org/project/azure-storage-file-datalake/

In [2]:
# Get connection string from: stg account Access Keys 
connection_string = "<connection_string>"

In [ ]:
# Show files/folders in container
from azure.storage.filedatalake import FileSystemClient

file_system = FileSystemClient.from_connection_string(
    connection_string, 
    file_system_name="mycont")

paths = file_system.get_paths()

for path in paths:
    print(path.name + '\n')

In [ ]:
# Read a file
from azure.storage.filedatalake import DataLakeFileClient
from io import StringIO

file = DataLakeFileClient.from_connection_string(connection_string,
                                                 file_system_name="cont2", file_path="frutas.csv")


data = file.download_file().read().decode('utf-8')
print(data)
df = pd.read_csv(StringIO(data))
df

In [ ]:
s_buf = StringIO()
df.to_csv(s_buf)

file = DataLakeFileClient.from_connection_string(connection_string,
                                                 file_system_name="cont2", file_path="frutas2.csv")
file.create_file()
file.append_data(data, offset=0)
file.flush_data(len(data))

### With SDK - gen 1

In [ ]:
# !pip install azure-mgmt-resource
# !pip install azure-mgmt-datalake-store
# !pip install azure-datalake-store

In [ ]:
# !pip list | grep 'azure-datalake-store\|azure-mgmt-datalake-store\|azure-mgmt-resource'

In [ ]:
from azure.datalake.store import core, lib, multithread
# https://github.com/Azure/azure-data-lake-store-python

In [ ]:
from azure.common.credentials import ServicePrincipalCredentials
# In entra create a new app and for that app a secret

TENANT_ID = '<tenant_id>' 
CLIENT_SECRET = '<clinet_secret>' # client secret value
CLIENT_ID = '<client_id>' # app id

token = lib.auth(tenant_id=TENANT_ID,
                 client_secret=CLIENT_SECRET,
                 client_id=CLIENT_ID)
token.token

In [ ]:
adlsFileSystemClient = core.AzureDLFileSystem(token, store_name='mystrgaccricgen1')
adlsFileSystemClient

# # Read a file into pandas dataframe
with adlsFileSystemClient.open('/contg1/frutas.csv', 'rb') as f:
    df = pd.read_csv(f) 

# # Show the dataframe
df

## Tables

In [ ]:
# !pip install azure-cosmosdb-table
# https://learn.microsoft.com/en-us/azure/cosmos-db/table/quickstart-python

In [25]:
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity

In [32]:
table_service = TableService(account_name='<storage_account_name>', account_key='<storage_account_key>')

In [ ]:
# create table
table_service.create_table('mytable2')

In [ ]:
# read table
def readTable(tableName):
    tasks = table_service.query_entities(tableName)
    tab = []
    newrow = []
    for row in tasks:
        for ele in row:
            newrow.append(row[ele])
        tab.append(newrow)
        newrow = []
    return tab

table = readTable('mytable')
print(table)

df = pd.DataFrame(table)
df

In [43]:
# write table
def setTable(tablename, table, table_service):
    index = 0
    for row in table:
        task = {'PartitionKey': "P"+str(index), 'RowKey': "R"+str(index+1)}
        index = index + 1
        for ele in row:
            task["Row"+str(row.index(ele))] = ele
        table_service.insert_entity(tablename, task)
    return True


df = pd.DataFrame([[1, 2, 3], [4, 5, 6], [8, 7, 9]])
list_of_lists = df.values.tolist()

# inserting csv to cloud
res = setTable('mytable', list_of_lists, table_service)